<a href="https://colab.research.google.com/github/dhanvithshetty-in/ai-agents-for-beginners/blob/main/Copy_of_Asana_Logic_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
from datetime import datetime


CSV_DATA = """Task ID,Created At,Completed At,Last Modified,Name,Section/Column,Assignee,Assignee Email,Start Date,Due Date,Tags,Notes,Projects,Parent task,Blocked By (Dependencies),Blocking (Dependencies),Priority,Task Progress
1211426128233325,2025-09-22,,2025-09-22,Gather system requirements,Planning,,,2025-09-17,2025-10-02,,,Engineering project plan,,,,High,
1211426128233348,2025-09-22,,2025-09-22,Meet stakeholders,,,,,,,,,Gather system requirements,,,,
1211426128233349,2025-09-22,,2025-09-22,Document business needs,,,,,,,,,Gather system requirements,,,,
1.21E+15,2025-09-22,,2025-09-22,Finalize requirement specification,,,,,,,,,Gather system requirements,,,,
1211426128233327,2025-09-22,,2025-09-22,Design database schema,Planning,,,2025-10-02,2025-10-08,,,Engineering project plan,,,,Medium,
1211426128233351,2025-09-22,,2025-09-22,Identify entities and relationships,Design database schema,,,,,,,,,,NaN,
1211426128233352,2025-09-22,,2025-09-22,Create ER diagram,Design database schema,,,,,,,,,,NaN,
1211426128233353,2025-09-22,,2025-09-22,Define primary & foreign keys,Design database schema,,,,,,,,,,NaN,
1211426128233331,2025-09-22,,2025-09-22,Develop user authentication module,In Progress,,,,,,,Engineering project plan,,,,Low,
1211426128233333,2025-09-22,,2025-09-22,Set up cloud infrastructure,In Progress,,,,,,,Engineering project plan,,,,High,
"""

class AsanaProjectSimulator:

    def __init__(self, csv_data):

        self.df = pd.read_csv(io.StringIO(csv_data))
        self.df['Start Date'] = pd.to_datetime(self.df['Start Date'], errors='coerce')
        self.df['Due Date'] = pd.to_datetime(self.df['Due Date'], errors='coerce')
        self.df['Completed At'] = pd.to_datetime(self.df['Completed At'], errors='coerce')
        self.df['Task ID'] = self.df['Task ID'].astype(str)
        print("✅ Project data loaded and initialized.")

    def get_effective_start_date(self, task_id):
        task = self.df[self.df['Task ID'] == task_id]
        if task.empty:
            return None, "Task not found"

        task = task.iloc[0]

        if pd.notna(task['Start Date']):
            return task['Start Date'], "using its own start date"

        if pd.notna(task['Parent task']):
            parent_name = task['Parent task']
            parent_task = self.df[self.df['Name'] == parent_name]
            if not parent_task.empty and pd.notna(parent_task.iloc[0]['Start Date']):
                return parent_task.iloc[0]['Start Date'], f"inheriting from parent '{parent_name}'"

        return None, "no start date found for task or parent"

    def mark_task_complete(self, task_id, completion_date_str):
        task_id = str(task_id)
        completion_date = datetime.strptime(completion_date_str, '%Y-%m-%d')

        task_row = self.df[self.df['Task ID'] == task_id]
        if task_row.empty:
            print(f"\nTask with ID {task_id} not found. Cannot mark as complete.")
            return False

        task_name = task_row.iloc[0]['Name']
        print(f"\nAttempting to complete '{task_name}' on {completion_date_str}...")

        effective_start_date, reason = self.get_effective_start_date(task_id)

        if effective_start_date:
            print(f"   - Effective start date is {effective_start_date.strftime('%Y-%m-%d')} ({reason}).")
            if completion_date.date() < effective_start_date.date():
                print(f"   - ❌ FAILED: Cannot complete task. Completion date ({completion_date_str}) is before start date ({effective_start_date.strftime('%Y-%m-%d')}).")
                return False
        else:
            print(f"   - ⚠️ SKIPPED CHECK: {reason}. Allowing completion.")

        self.df.loc[self.df['Task ID'] == task_id, 'Completed At'] = completion_date
        self.df.loc[self.df['Task ID'] == task_id, 'Section/Column'] = 'Completed'
        print(f"   - ✅ SUCCESS: Task '{task_name}' marked as complete.")
        return True

    def display_project_status(self):
        print("\n--- Current Project Status ---")
        print(self.df[['Name', 'Parent task', 'Start Date', 'Completed At', 'Section/Column']].to_string())
        print("-----------------------------\n")


if __name__ == "__main__":
    project = AsanaProjectSimulator(CSV_DATA)
    project.display_project_status()

    project.mark_task_complete('1211426128233348', '2025-09-16')

    project.mark_task_complete('1211426128233348', '2025-09-17')

    project.mark_task_complete('1211426128233352', '2025-10-01')

    project.mark_task_complete('1211426128233352', '2025-10-03')

    project.mark_task_complete('1211426128233331', '2025-09-25')

    project.display_project_status()

✅ Project data loaded and initialized.

--- Current Project Status ---
                                  Name                 Parent task Start Date Completed At          Section/Column
0           Gather system requirements                         NaN 2025-09-17          NaT                Planning
1                    Meet stakeholders  Gather system requirements        NaT          NaT                     NaN
2              Document business needs  Gather system requirements        NaT          NaT                     NaN
3   Finalize requirement specification  Gather system requirements        NaT          NaT                     NaN
4               Design database schema                         NaN 2025-10-02          NaT                Planning
5  Identify entities and relationships                         NaN        NaT          NaT  Design database schema
6                    Create ER diagram                         NaN        NaT          NaT  Design database schema
7        